[Sentiment analysis on Twitter using word2vec and keras](https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html)

In [1]:
from copy import deepcopy
from string import punctuation
from random import shuffle

import pandas as pd
import numpy as np
import gensim
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [2]:
pd.options.mode.chained_assignment = None
LabeledSentence = gensim.models.doc2vec.LabeledSentence
tqdm.pandas(desc="progress-bar")
tokenizer = TweetTokenizer()

In [3]:
data = pd.read_csv('dados/tweets.csv', 
                names=['Sentiment', 'a', 'b', 'c', 
                'd','SentimentText'], 
                header=None,
                nrows=1000000)

In [4]:
data.tail()

,Sentiment,a,b,c,d,SentimentText
999995,4,1879942807,Thu May 21 23:36:19 PDT 2009,NO_QUERY,divabat,"@healingsinger thank you, i needed that"
999996,4,1879942922,Thu May 21 23:36:20 PDT 2009,NO_QUERY,nick1975,@vactress http://bit.ly/cADea Maybe this is m...
999997,4,1879942975,Thu May 21 23:36:21 PDT 2009,NO_QUERY,znmeb,"@Brat13 Hell, Windows 7 will be out of my pric..."
999998,4,1879943113,Thu May 21 23:36:22 PDT 2009,NO_QUERY,virmani,@jigardoshi neah.. i wish! just reminiscing r...
999999,4,1879943219,Thu May 21 23:36:24 PDT 2009,NO_QUERY,redcomet81,@MsTeagan ...and by the way: I rewatched Sun G...


In [5]:
data.drop(['a', 'b', 'c', 'd'], axis=1, inplace=True)

In [6]:
data.head()

,Sentiment,SentimentText
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [35]:
# data['Sentiment'] = data['Sentiment'].map(int)
data['Sentiment'] = data['Sentiment'].map( {4:1, 0:0} )

In [36]:
data = data[data['SentimentText'].isnull() == False]

In [37]:
data.size

2998791

In [38]:
def tokenize(tweet):
    try:
        tweet = unicode(tweet.decode('utf-8').lower())
        tokens = tokenizer.tokenize(tweet)
        tokens = filter(lambda t: not t.startswith('@'), tokens)
        tokens = filter(lambda t: not t.startswith('#'), tokens)
        tokens = filter(lambda t: not t.startswith('http'), tokens)
        return tokens
    except:
        return 'NC'

In [39]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data['tokens'] = data['SentimentText'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [40]:
data = postprocess(data)

progress-bar: 100%|██████████| 999597/999597 [01:54<00:00, 8718.10it/s]


In [41]:
data.tail()

,Sentiment,SentimentText,tokens
999592,1,"@healingsinger thank you, i needed that","[thank, you, ,, i, needed, that]"
999593,1,@vactress http://bit.ly/cADea Maybe this is m...,"[maybe, this, is, more, you]"
999594,1,"@Brat13 Hell, Windows 7 will be out of my pric...","[hell, ,, windows, 7, will, be, out, of, my, p..."
999595,1,@jigardoshi neah.. i wish! just reminiscing r...,"[neah, .., i, wish, !, just, reminiscing, read..."
999596,1,@MsTeagan ...and by the way: I rewatched Sun G...,"[..., and, by, the, way, :, i, rewatched, sun,..."


# Criando o modelo word2vec

In [42]:
x_train, x_test, y_train, y_test = train_test_split(np.array(data.tokens),
                                                    np.array(data.Sentiment), test_size=0.2)

In [43]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

In [44]:
x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

0it [00:00, ?it/s]/home/cloves/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
799677it [00:06, 125281.92it/s]
199920it [00:01, 156021.24it/s]


In [45]:
x_train[0]

TaggedDocument(words=[u'download', u'movie', u'"', u'the', u'lady', u'vanishes', u'"', u'cool'], tags=['TRAIN_0'])

In [46]:
tweet_w2v = Word2Vec(size=200, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])

100%|██████████| 799677/799677 [00:00<00:00, 1334556.89it/s]


In [47]:
tweet_w2v.train([x.words for x in tqdm(x_train)], total_examples=tweet_w2v.corpus_count, epochs=tweet_w2v.iter)

100%|██████████| 799677/799677 [00:00<00:00, 1231778.93it/s]


42946361

In [48]:
tweet_w2v['good']

/home/cloves/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


array([-1.90684736,  1.01301861,  0.32150444,  1.04957163, -0.58150125,
        0.54862761,  1.63492012, -0.12029213, -0.74207795, -0.67328894,
        1.19723296,  2.20949006, -0.20067705, -0.83516425, -1.21178496,
        0.10092524,  0.19687739, -0.0798308 ,  1.02091777,  1.98775125,
        0.40227008,  0.07540293,  0.59218061,  0.76983839,  0.20866959,
        1.08864212,  1.06473184, -0.33474469, -0.98856843, -1.39319754,
        0.14923029,  1.05066597, -0.80196804, -1.24420512, -0.95128745,
       -0.62479472, -1.03385472,  0.79990554, -0.5806008 , -1.00982416,
       -0.40763941,  0.88685769,  0.93947047,  0.99475646,  0.42686781,
        0.71891046,  0.61988688,  1.07250226, -2.93749261, -1.15797484,
       -0.62610322,  1.63865364, -1.44832087,  0.97315454, -0.88633025,
        0.75043738,  0.13194846,  2.57319713, -1.39985168, -0.51340926,
        0.31402341, -1.36632431, -0.80903739, -0.30807737,  0.69173414,
        2.59744072,  1.95941722,  0.97506696,  0.5799349 ,  0.07

In [49]:
tweet_w2v.most_similar('good')

/home/cloves/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[(u'goood', 0.7293262481689453),
 (u'great', 0.696771502494812),
 (u'fantastic', 0.6293584704399109),
 (u'rough', 0.6238858103752136),
 (u'pleasant', 0.620288610458374),
 (u'nice', 0.615257978439331),
 (u'gd', 0.6050779819488525),
 (u'tough', 0.6037716865539551),
 (u'bad', 0.6011664867401123),
 (u'terrible', 0.5841963887214661)]

In [50]:
tweet_w2v.most_similar('bar')

/home/cloves/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[(u'table', 0.7307553291320801),
 (u'pub', 0.7164860963821411),
 (u'restaurant', 0.6950686573982239),
 (u'ranch', 0.6935731172561646),
 (u'cafe', 0.6768673062324524),
 (u'casino', 0.661666214466095),
 (u'gate', 0.6539527773857117),
 (u'grill', 0.6453628540039062),
 (u'club', 0.6190779209136963),
 (u'subway', 0.6163418292999268)]

How about visualizing these word vectors? We first have to reduce their dimension to 2 using t-SNE. Then, using an interactive visualization tool such as Bokeh, we can map them directly on 2D plane and interact with them.
Here's the script, and the bokeh chart below.

In [51]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [52]:
# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

Loading BokehJS ...

In [53]:
# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [tweet_w2v[w] for w in tweet_w2v.wv.vocab.keys()[:5000]];

/home/cloves/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [54]:
# dimensionality reduction. converting the vectors to 2d vectors

from sklearn.manifold import TSNE

'''
tsne_model = TSNE(n_components=2, verbose=1, 
                  random_state=0, perplexity=30, n_iter=300)
'''
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.165s...
[t-SNE] Computed neighbors for 5000 samples in 10.708s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.269329
[t-SNE] KL divergence after 250 iterations with early exaggeration: 88.949928
[t-SNE] Error after 1000 iterations: 2.894572


In [55]:
# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = tweet_w2v.wv.vocab.keys()[:5000]

In [56]:
tsne_df.head()

,x,y,words
0,-8.496893,9.257027,patd
1,26.194971,-27.252241,woods
2,23.827072,-15.693091,spiders
3,38.432861,-16.617390,hanging
4,10.880193,21.218609,woody


In [57]:
from bokeh.models import ColumnDataSource

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', 
                   y='y', 
                   source=ColumnDataSource(tsne_df))
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

# Criando um classificador de sentimentos

In [58]:
print 'building tf-idf matrix ...'
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print 'vocab size :', len(tfidf)

building tf-idf matrix ...
vocab size : 23057


In [59]:
# Now let's define a function that, given a list of tweet tokens, creates an averaged tweet vector.
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [60]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

100%|██████████| 199920/199920 [10:41<00:00, 311.85it/s]


We should now be ready to feed these vectors into a neural network classifier. In fact, using Keras is very easy to define layers and activation functions.
Here is a basic 2-layer architecture.

In [61]:
from keras.models import Sequential
from keras.layers import Dense, Activation


model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=9, batch_size=32, verbose=2)

Epoch 1/9
 - 35s - loss: 0.3491 - acc: 0.8545
Epoch 2/9
 - 33s - loss: 0.3368 - acc: 0.8596
Epoch 3/9
 - 29s - loss: 0.3336 - acc: 0.8611
Epoch 4/9
 - 28s - loss: 0.3317 - acc: 0.8617
Epoch 5/9
 - 28s - loss: 0.3305 - acc: 0.8620
Epoch 6/9
 - 28s - loss: 0.3294 - acc: 0.8626
Epoch 7/9
 - 28s - loss: 0.3287 - acc: 0.8633
Epoch 8/9
 - 28s - loss: 0.3280 - acc: 0.8634
Epoch 9/9
 - 28s - loss: 0.3274 - acc: 0.8636


In [62]:
score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=2)
print score[1]

0.859323729497
